반복적인 디비 불러오는건 피한다. <- 지능형 

비정형 데이터 <- 일반적으로 db안씀 파일시스템 트리구조로 사용 

orm -> class와 table 연결(연관성)


In [1]:
import sqlite3

In [2]:
sqlite3.version # 패키지 버전

'2.6.0'

In [3]:
sqlite3.version_info # 런타임 돌아가는 버전

(2, 6, 0)

In [4]:
type(sqlite3.version)

str

In [5]:
type(sqlite3.version_info)

tuple

In [11]:
conn = sqlite3.connect(':memory:')

In [12]:
conn

In [13]:
type(conn)

sqlite3.Connection

In [14]:
cur = conn.cursor()

In [15]:
cur

In [16]:
type(cur)

sqlite3.Cursor

In [17]:
dir(cur)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'arraysize',
 'close',
 'connection',
 'description',
 'execute',
 'executemany',
 'executescript',
 'fetchall',
 'fetchmany',
 'fetchone',
 'lastrowid',
 'row_factory',
 'rowcount',
 'setinputsizes',
 'setoutputsize']

In [20]:
cur.execute("create table people (name_last , age)")

In [21]:
who = "Yeltsin"

In [22]:
age = 72

In [23]:
cur.execute("insert into people values (?, ?)", (who, age)) # 변수 사용

In [27]:
cur.execute("select * from people where name_last=:who and age=:age",{"who":who, "age":age})
# dict 사용

In [30]:
print(cur.fetchone()) # 물고 있는거 하나씩 가져온다

None


In [31]:
type(cur.fetchone())

NoneType

exeutemany

In [39]:
sql = "insert into people values( ?, ?)" # sql string; ?사용

In [40]:
data = [("1", 1), ("2",2), ("3",3)]

In [41]:
cur.executemany(sql,data) # data크기만큼 반복을 해준다

In [35]:
cur.executescript(
"""
    create table person (
        first_name text primary key, 
        last_name text not null
    );
    
    insert into person values('name','kim');
"""
    
) # sql문 한번에 여러개 select문 쓰면 못 읽어

In [44]:
cur.execute("select * from people")

In [45]:
cur.fetchall()

[('Yeltsin', 72), ('1', 1), ('2', 2), ('3', 3), ('1', 1), ('2', 2), ('3', 3)]

In [46]:
conn.close()

In [47]:
conn = sqlite3.connect("test.db")

In [62]:
conn.execute(
'''
    create table company(
        id int primary key not null,
        name text not null,
        age int not null,
        address char(50),
        salary real);
'''
    
)

In [56]:
conn.execute('drop table company')

In [64]:
conn.execute("insert into company (id, name, age, address, salary) \
               values (1, 'paul',32,'california',20000.00)")


In [67]:
conn.execute("insert into company values(:id, :name, :age, :address, :salary )",
            {'id':2, 'name':'Allen', 'age':25, 'address':'Texas', 'salary':150000.00})

In [68]:
data = [(3, 'Teddy', 23, 'Norway',200000.00),
        (4, 'Mark', 25, 'Rich-Mond', 6500.00)]

In [69]:
conn.executemany("insert into company values(?,?,?,?,?)",data)

In [70]:
conn.executescript("""
    insert into company values (5, 'Mark1', 25, 'Rich-Mond',65000.00 );
    insert into company values (6, 'Mark2', 25, 'Rich-Mond',65000.00 );
""")

In [71]:
conn.commit()   # 하나의 트렌젝션 finish 
                # 많은 sql문을 하였을 때 반영을 하기 위해 commit
                # 너무 많이하면 오브젝트가 커진다
                # select를 할때 그 전에 commit을 수행해줘야한다. update전에도 동일

In [72]:
# qmarker
# namspace
# 리스트 형식

In [79]:
temp = conn.execute("select * from company")

In [76]:
type(temp) # cursor를 execute가 가져온거야

sqlite3.Cursor

In [80]:
for row in temp:
    print(type(row),row) # 패치 한결과와 똑같이 나온다.

<class 'tuple'> (1, 'paul', 32, 'california', 20000.0)
<class 'tuple'> (2, 'Allen', 25, 'Texas', 150000.0)
<class 'tuple'> (3, 'Teddy', 23, 'Norway', 200000.0)
<class 'tuple'> (4, 'Mark', 25, 'Rich-Mond', 6500.0)
<class 'tuple'> (5, 'Mark1', 25, 'Rich-Mond', 65000.0)
<class 'tuple'> (6, 'Mark2', 25, 'Rich-Mond', 65000.0)


In [82]:
cid = 1 

In [83]:
conn.execute("update company set salary = 25000.00 where id = :id ",{'id':cid})

In [85]:
conn.commit() # select, update 는 commit을 끝내줘여한다.

In [86]:
conn.execute("delete from company where id = 2")

In [88]:
conn.commit()# 반영을 해줘야한다. slelect문으로 하면 트랜젝션을 끝내서 똑같다 .(but!! 경우에 따라 달라서 commit을 해줘)
